### 1. Data Comprehension and Visualization

In [1]:
import pandas as pd

df = pd.read_csv('original_news_dataset.csv')
df.head()

,Id,date,platform,title,News content,Label
0,1,2023-01-11 00:00:00,Aljazeera,الضفة الغربية.. الاحتلال يهدم 17 منزلا تاريخيا...,هدمت قوات الاحتلال الإسرائيلي -اليوم الأربعاء-...,real
1,2,2023-01-11 00:00:00,Aljazeera,مظاهرات بمدن أوروبية تضامنا مع غزة وحشود أمام ...,خرجت مظاهرات في عدد من المدن الأوروبية مساء ال...,real
2,3,2023-01-11 00:00:00,Aljazeera,شهداء في جنين وطولكرم وإضراب عام بالضفة الغربي...,استشهد 4 فلسطينيين واعتقل عشرات آخرون -اليوم ا...,real
3,4,2023-02-11 00:00:00,Aljazeera,أبو عبيدة: خسائر العدو أكبر بكثير مما يعلن وسن...,أكد الناطق باسمكتائب الشهيد عز الدين القسام-ال...,real
4,5,2023-03-11 00:00:00,Aljazeera,9 شهداء بالضفة والاحتلال يشن حملة اعتقالات,استشهد 9 فلسطينيين في مواجهات اندلعت مع قوات ا...,real


In [2]:
df.shape

(5352, 6)

In [3]:
# Check value counts for the 'Label' column
label_counts = df['Label'].value_counts()
label_counts

Label
real    3913
fake    1439
Name: count, dtype: int64

The dataset is successfully loaded and contains **5,352** news articles with the following columns:

- **Id**: Unique identifier  
- **date**: Date of publication  
- **platform**: News source (e.g., *Aljazeera*)  
- **title**: Article headline  
- **News content**: Full news body (*Arabic*)  
- **Label**: Either *real* or *fake*


In [4]:
# Get platform value counts
platform_counts = df['platform'].value_counts()
platform_counts


platform
Aljazeera                        3422
Misbar                           1426
Tibyan                            247
X                                  64
الناشط حمزة المصري                 37
قناة النصيرات الإخبارية            37
Facebook                           17
AFP                                15
الاتحاد الفلسطيني لكرة القدم       15
Reuters                            15
وزارة التربية والتعليم العالي      10
وزارة التنمية الاجتماعية            9
سلطة جودة البيئة                    9
YouTube                             6
وزارة الخارجية والمغتربين           5
Instagram                           5
TikTok                              2
Reuters Fact Check                  2
AFP Fact Check                      2
1                                   1
وزارة الثقافة                       1
وزارة الصحة الفلسطينية              1
Twitter                             1
chayyek                             1
tayqan                              1
معبر كيسوفيم يستقبل 300 شاحنة       1
Nam

In [5]:
import plotly.express as px

# Step 1: Get top 3 platforms
top_3_platforms = df['platform'].value_counts().nlargest(3).index

# Step 2: Replace all other platforms with 'Other'
df['platform_grouped'] = df['platform'].apply(lambda x: x if x in top_3_platforms else 'Other')

# Step 3: Count articles per grouped platform
platform_counts_grouped = df['platform_grouped'].value_counts().reset_index()
platform_counts_grouped.columns = ['Platform', 'Number of Articles']

# Step 4: Plot using Plotly
fig = px.bar(
    platform_counts_grouped,
    x='Platform',
    y='Number of Articles',
    title='Articles per Platform (Top 3 + Other)',
    color='Platform',
    text='Number of Articles'
)

fig.update_layout(
    xaxis_title='Platform',
    yaxis_title='Number of Articles',
    xaxis_tickangle=0,
    showlegend=False
)

fig.update_traces(textposition='outside')

fig.show()


In [6]:
print("📊 Top 3 Platforms + Others:")
print(platform_counts_grouped)

📊 Top 3 Platforms + Others:
    Platform  Number of Articles
0  Aljazeera                3422
1     Misbar                1426
2      Other                 257
3     Tibyan                 247


**Others**: These are the remaining news channels. Although they are numerous, each contributes only a small number of articles. Therefore, they were grouped together for graphical representation purposes.


In [7]:
# Total duplicates
total_duplicates = df.duplicated().sum()
null_counts = df.isnull().sum()

print(f"Total duplicate rows: " ,total_duplicates)
print("Null values per column:", null_counts)


Total duplicate rows:  0
Null values per column: Id                  0
date                0
platform            0
title               0
News content        0
Label               0
platform_grouped    0
dtype: int64


In [8]:
df['title_length'] = df['title'].astype(str).str.len()
df['content_length'] = df['News content'].astype(str).str.len()

# Basic statistics for title and content length
print("Title Length Stats:")
print(df['title_length'].describe())

print("\nNews Content Length Stats:")
print(df['content_length'].describe())


Title Length Stats:
count    5352.000000
mean       55.764948
std        27.468840
min         7.000000
25%        35.000000
50%        56.000000
75%        69.000000
max       379.000000
Name: title_length, dtype: float64

News Content Length Stats:
count     5352.000000
mean      1363.517003
std       2523.191387
min          7.000000
25%        119.000000
50%        268.000000
75%       2003.000000
max      64878.000000
Name: content_length, dtype: float64


In [9]:
# 1. Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Handle invalid dates gracefully

# 2. Create new columns for year and month
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['month_name'] = df['date'].dt.month_name()

# 3. Combine year and month into a single label (optional)
df['year_month'] = df['date'].dt.to_period('M').astype(str)

# 4. Count the number of articles per month
month_distribution = df['year_month'].value_counts().sort_index()

# 5. Display the most common month
most_common_month = month_distribution.idxmax()
most_common_count = month_distribution.max()

print("Article Distribution by Month:")
print(month_distribution)

Article Distribution by Month:
year_month
2011-03      1
2015-06      1
2016-06      1
2017-09      1
2019-04      1
2019-07      1
2020-05      2
2020-09      1
2020-12      1
2021-04      2
2021-07      1
2022-01      2
2022-04      1
2022-09      1
2023-01      8
2023-02     10
2023-03      3
2023-04     16
2023-05     21
2023-06     32
2023-07     60
2023-08     61
2023-09     19
2023-10    185
2023-11    289
2023-12    588
2024-01    413
2024-02    115
2024-03     11
2024-04    105
2024-05    301
2024-06     10
2024-07    333
2024-08    382
2024-09    342
2024-10    395
2024-11    350
2024-12    337
2025-01      8
2025-04    400
2025-05    364
2025-06    177
Name: count, dtype: int64


In [10]:
import plotly.express as px
import pandas as pd

# Step 1: Prepare the data (ensure it's a DataFrame)
month_distribution_df = month_distribution.reset_index()
month_distribution_df.columns = ['Year-Month', 'Number of Articles']

# Step 2: Plot using Plotly
fig = px.bar(
    month_distribution_df,
    x='Year-Month',
    y='Number of Articles',
    title='News Articles per Month',
    text='Number of Articles'
)

# Step 3: Customize layout
fig.update_layout(
    xaxis_title='Year-Month',
    yaxis_title='Number of Articles',
    xaxis_tickangle=45
)

fig.update_traces(textposition='outside')

# Step 4: Show the figure
fig.show()


In [11]:
import plotly.express as px
import pandas as pd

# Step 1: Prepare the data
label_counts_df = df['Label'].value_counts().reset_index()
label_counts_df.columns = ['Label', 'Number of Articles']

# Step 2: Plot using Plotly
fig = px.bar(
    label_counts_df,
    x='Label',
    y='Number of Articles',
    color='Label',
    title='Real vs Fake News Count',
    text='Number of Articles',
    color_discrete_map={'real': 'green', 'fake': 'red'}  # Optional: fixed color map
)

# Step 3: Customize layout
fig.update_layout(
    xaxis_title='News Label',
    yaxis_title='Number of Articles',
    xaxis_tickangle=0,
    showlegend=False
)

fig.update_traces(textposition='outside')

# Step 4: Show the figure
fig.show()


In [12]:
import plotly.express as px

# Step 1: Make sure 'title_length' is calculated
df['title_length'] = df['title'].astype(str).str.len()

# Step 2: Plot using Plotly
fig = px.histogram(
    df,
    x='title_length',
    nbins=30,
    title='📝 Title Length Distribution',
    labels={'title_length': 'Number of Characters in Title'},
)

# Step 3: Customize layout
fig.update_layout(
    xaxis_title='Number of Characters in Title',
    yaxis_title='Number of Articles',
    bargap=0.1
)

fig.show()


In [13]:
import plotly.express as px

# Step 1: Make sure 'content_length' is calculated
df['content_length'] = df['News content'].astype(str).str.len()

# Step 2: Plot using Plotly
fig = px.histogram(
    df,
    x='content_length',
    nbins=50,
    title='📄 News Content Length Distribution',
    labels={'content_length': 'Number of Characters in News Content'}
)

# Step 3: Customize layout
fig.update_layout(
    xaxis_title='Number of Characters in News Content',
    yaxis_title='Number of Articles',
    bargap=0.1
)

# Step 4: Show the plot
fig.show()


### 2. Data Cleaning 

In [14]:
columns_to_drop = [
    'Id', 'date', 'platform', 'platform_grouped', 'title_length', 
    'content_length', 'year', 'month', 'month_name', 
    'year_month', 'tokens', 'filtered_tokens'
]

# Drop the columns if they exist
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)
df

,title,News content,Label
0,الضفة الغربية.. الاحتلال يهدم 17 منزلا تاريخيا...,هدمت قوات الاحتلال الإسرائيلي -اليوم الأربعاء-...,real
1,مظاهرات بمدن أوروبية تضامنا مع غزة وحشود أمام ...,خرجت مظاهرات في عدد من المدن الأوروبية مساء ال...,real
2,شهداء في جنين وطولكرم وإضراب عام بالضفة الغربي...,استشهد 4 فلسطينيين واعتقل عشرات آخرون -اليوم ا...,real
3,أبو عبيدة: خسائر العدو أكبر بكثير مما يعلن وسن...,أكد الناطق باسمكتائب الشهيد عز الدين القسام-ال...,real
4,9 شهداء بالضفة والاحتلال يشن حملة اعتقالات,استشهد 9 فلسطينيين في مواجهات اندلعت مع قوات ا...,real
...,...,...,...
5347,"الحرب على غزة مباشر.. مجازر إسرائيلية بمراكز ""...",في اليوم الـ77 من استئناف حرب الإبادة على غزة،...,real
5348,الصورة ليست لجندي مغربي سقط في كمين للمقاومة ا...,تتداول حسابات على موقع التواصل الاجتماعي إكس، ...,fake
5349,مظاهرات بإسرائيل تطالب بإكمال الصفقة وجدل حول ...,تظاهر محتجون وسط تل أبيب ليل الاثنين وأضرموا ا...,real
5350,"زكريا الزبيدي حرا.. ""التنين"" الفلسطيني الذي هز...",لم يكن في مخيلة وزير الأمن القومي الإسرائيلي ا...,real


In [15]:
df['text'] = df['title'] + ' ' + df['News content']
df['text']

0       الضفة الغربية.. الاحتلال يهدم 17 منزلا تاريخيا...
1       مظاهرات بمدن أوروبية تضامنا مع غزة وحشود أمام ...
2       شهداء في جنين وطولكرم وإضراب عام بالضفة الغربي...
3       أبو عبيدة: خسائر العدو أكبر بكثير مما يعلن وسن...
4       9 شهداء بالضفة والاحتلال يشن حملة اعتقالات است...
                              ...                        
5347    الحرب على غزة مباشر.. مجازر إسرائيلية بمراكز "...
5348    الصورة ليست لجندي مغربي سقط في كمين للمقاومة ا...
5349    مظاهرات بإسرائيل تطالب بإكمال الصفقة وجدل حول ...
5350    زكريا الزبيدي حرا.. "التنين" الفلسطيني الذي هز...
5351    فيديو قديم لمعتمر يدعو لغزة خلال طوافه في الحر...
Name: text, Length: 5352, dtype: object

In [16]:
import nltk
import re
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Arabic-specific text cleaning and tokenization
def arabic_preprocess(text):
    # Normalize Arabic text
    text = re.sub(r'[^\u0600-\u06FF\s]', ' ', str(text))  # keep Arabic chars only
    text = re.sub(r'\s+', ' ', text).strip()
    return word_tokenize(text)

df['tokens'] = df['text'].apply(arabic_preprocess)
df[['text', 'tokens']].head()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,text,tokens
0,الضفة الغربية.. الاحتلال يهدم 17 منزلا تاريخيا...,"[الضفة, الغربية, الاحتلال, يهدم, منزلا, تاريخي..."
1,مظاهرات بمدن أوروبية تضامنا مع غزة وحشود أمام ...,"[مظاهرات, بمدن, أوروبية, تضامنا, مع, غزة, وحشو..."
2,شهداء في جنين وطولكرم وإضراب عام بالضفة الغربي...,"[شهداء, في, جنين, وطولكرم, وإضراب, عام, بالضفة..."
3,أبو عبيدة: خسائر العدو أكبر بكثير مما يعلن وسن...,"[أبو, عبيدة, خسائر, العدو, أكبر, بكثير, مما, ي..."
4,9 شهداء بالضفة والاحتلال يشن حملة اعتقالات است...,"[شهداء, بالضفة, والاحتلال, يشن, حملة, اعتقالات..."


In [17]:
from nltk.corpus import stopwords
nltk.download('stopwords')

arabic_stopwords = set(stopwords.words('arabic'))

df['filtered_tokens'] = df['tokens'].apply(lambda tokens: [w for w in tokens if w not in arabic_stopwords and len(w) > 1])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
df.drop(columns=['title', 'News content','text','tokens'], inplace=True)

In [19]:
desired_order = ['filtered_tokens', 'Label']
df = df[desired_order]

In [20]:
df.rename(columns={'filtered_tokens': 'processed_text'}, inplace=True)


In [21]:
df.to_csv("cleaned_news_dataset.csv", index=False)

In [22]:
df2 = pd.read_csv('cleaned_news_dataset.csv')
df2.head()

,processed_text,Label
0,"['الضفة', 'الغربية', 'الاحتلال', 'يهدم', 'منزل...",real
1,"['مظاهرات', 'بمدن', 'أوروبية', 'تضامنا', 'غزة'...",real
2,"['شهداء', 'جنين', 'وطولكرم', 'وإضراب', 'عام', ...",real
3,"['عبيدة', 'خسائر', 'العدو', 'أكبر', 'بكثير', '...",real
4,"['شهداء', 'بالضفة', 'والاحتلال', 'يشن', 'حملة'...",real


In [23]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer


# Step 1: Convert token lists (stored as strings) back into actual text
df2['processed_text_str'] = df2['processed_text'].apply(lambda x: ' '.join(eval(x)))

# Step 2: Define Features and Labels
X_text = df2['processed_text_str']
y = df2['Label'].apply(lambda label: 1 if label.lower() == 'real' else 0)  # 1 = real, 0 = fake

# Step 3: TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))  # unigrams + bigrams
X = vectorizer.fit_transform(X_text)

# Show shape of the TF-IDF matrix and label distribution
print("TF-IDF Matrix Shape:", X.shape)
print("Label Distribution:\n", y.value_counts())


TF-IDF Matrix Shape: (5352, 5000)
Label Distribution:
 Label
1    3913
0    1439
Name: count, dtype: int64


In [24]:
from sklearn.model_selection import train_test_split

# Split the data (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Display basic information
print("Training Set Shape:", X_train.shape)
print("Test Set Shape:", X_test.shape)
print("\nLabel Distribution in Training Set:\n", y_train.value_counts())
print("\nLabel Distribution in Test Set:\n", y_test.value_counts())


Training Set Shape: (4281, 5000)
Test Set Shape: (1071, 5000)

Label Distribution in Training Set:
 Label
1    3130
0    1151
Name: count, dtype: int64

Label Distribution in Test Set:
 Label
1    783
0    288
Name: count, dtype: int64


In [25]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Define the models to compare
models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Linear SVM": LinearSVC(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

# Train and evaluate each model
results = []

for name, model in models.items():
    model.fit(X_train, y_train)               # Train the model
    y_pred = model.predict(X_test)            # Predict on test set
    acc = accuracy_score(y_test, y_pred)      # Overall accuracy
    report = classification_report(y_test, y_pred, output_dict=True)
    
    results.append({
        "Model": name,
        "Accuracy": acc,
        "Precision (Fake)": report['0']['precision'],
        "Recall (Fake)": report['0']['recall'],
        "F1-Score (Fake)": report['0']['f1-score'],
        "Precision (Real)": report['1']['precision'],
        "Recall (Real)": report['1']['recall'],
        "F1-Score (Real)": report['1']['f1-score'],
    })

# Show results as a table
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="Accuracy", ascending=False)
print(results_df.to_string(index=False))


              Model  Accuracy  Precision (Fake)  Recall (Fake)  F1-Score (Fake)  Precision (Real)  Recall (Real)  F1-Score (Real)
      Random Forest  0.887955          0.785714       0.802083         0.793814          0.926641       0.919540         0.923077
         Linear SVM  0.878618          0.782143       0.760417         0.771127          0.912769       0.922095         0.917408
Logistic Regression  0.875817          0.818930       0.690972         0.749529          0.892512       0.943806         0.917443
        Naive Bayes  0.873950          0.793103       0.718750         0.754098          0.900000       0.931034         0.915254


In [26]:
# Make sure 'vectorizer' and 'model' (RandomForest) are already trained

# Sample Arabic news article
sample_text = """
🔴مصادر مطلعة للغد:
نص المقترح الأمريكي المعروض على حركة حماس:

وقف إطلاق نار لمدة 60 يومًا بضمانات من ترامب تضمن استمراره طوال المدة.

جدول الإفراج عن 10 أسرى أحياء و18 جثمانًا:
• اليوم الأول: 8 أسرى أحياء
• اليوم السابع: 5 جثامين
• اليوم الثلاثون: 5 جثامين
• اليوم الخمسون: 2 من الأسرى الأحياء
• اليوم الستون: 8 جثامين

المساعدات الإنسانية ستدخل فورًا وفقًا لاتفاق 19 يناير، وبكميات كافية، بمشاركة الأمم المتحدة والهلال الأحمر.
• في اليوم الأول، بعد الإفراج عن 8 أسرى، سينسحب الجيش الإسرائيلي من مناطق في شمال غزة، حسب خرائط يتم التوافق عليها.
• في اليوم السابع، بعد استلام 5 جثامين، ستتم عملية انسحاب إسرائيلية من مناطق في الجنوب، حسب خرائط متفق عليها.
• ستعمل فِرَق فنية على رسم حدود الانسحابات خلال مفاوضات سريعة تُجرى بعد الاتفاق على الإطار العام للمقترح.

⸻

في اليوم الأول من الاتفاق تبدأ مفاوضات حول وقف دائم لإطلاق النار، تتناول أربع نقاط رئيسية:
1. تبادل ما تبقى من الأسرى
2. الترتيبات الأمنية طويلة الأمد في غزة
3. ترتيبات “اليوم التالي”
4. إعلان وقف دائم لإطلاق النار

⸻

المقترح يتضمن ضمانات لالتزام ترامب وجديته تجاه الاتفاق، وأنه في حال نجاح المفاوضات خلال فترة التهدئة، فسيؤدي ذلك إلى نهاية دائمة للنزاع.
• ستجري عمليات تبادل الأسرى بدون احتفالات أو استعراضات.
• في اليوم العاشر، ستقدم حماس كل المعلومات والأدلة حول الأسرى المتبقين وما إذا كانوا أحياء أو أموات، مع تقارير طبية. وفي المقابل، ستقدم إسرائيل معلومات كاملة عن الأسرى الفلسطينيين الذين تم اعتقالهم من غزة منذ 7 أكتوبر 2023.

الوسطاء (مصر، قطر، الولايات المتحدة) سيقدمون ضمانات بأن مفاوضات جادة ستجري خلال فترة التهدئة، وإذا استدعى الأمر، يمكن تمديد تلك الفترة. وفي حال التوصل إلى اتفاق، سيتم إطلاق سراح جميع الأسرى المتبقين.

سيُعلِن ترامب بنفسه التوصل إلى الاتفاق، وكذلك التزام الولايات المتحدة بمواصلة المفاوضات لضمان وقف دائم لإطلاق النار.
وسيتولى ستيف ويتكوف قيادة المفاوضات لإنهاء الحرب
"""

# Preprocessing: clean and tokenize (if needed)
# For simplicity, let's assume it's already cleaned like your dataset
# Otherwise, use your own pre-processing pipeline here
processed = sample_text.split()  # simple tokenization
joined_text = " ".join(processed)

# Vectorize it using the same TF-IDF vectorizer
sample_vector = vectorizer.transform([joined_text])

# Predict
prediction = model.predict(sample_vector)

# Output result
label = "Real" if prediction[0] == 1 else "Fake"
print("Prediction:", label)


Prediction: Real
